In [ ]:
## imports
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, \
                            roc_auc_score, precision_score, f1_score, recall_score, \
                            mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from ast import literal_eval as make_tuple
from sklearn.pipeline import Pipeline
from datetime import timedelta
from time import time
import pandas as pd
import numpy as np
import sys

In [ ]:
sys.path.insert(1, "../../utils/")
import model_utils

results_dir = "./"
best_results_path = "../grid_search_results/dtree_params.csv"
best_results = pd.read_csv(best_results_path)

In [ ]:
# get best params according to grid_search best params
top3_best_params = best_results.nlargest(2,['model_accuracy', 'f1_score', 'roc_score'])
top3_best_params.drop_duplicates(["epoch_best_params"], inplace = True)

# list top 3 results in which epoch_best_params are different
top3_best_params.head(3)
#print(top3_best_params["epoch_best_params"].values)

In [ ]:
all_predictions = []
for row in top3_best_params.iterrows():
    # load correct database in which best results were saved
    dbname = row[1]["database"]
    dataframe_path = f"../../pre_processing/datasets/preprocessed_data/{dbname}.csv"
    df = pd.read_csv(dataframe_path, sep = ",")

    # Drop RA column if exists
    if "RA" in df.columns:
        print("Removing column RA")
        df.drop('RA', inplace=True, axis=1)

    X = df.iloc[:, df.columns != "EVADIDO"].values
    y = df["EVADIDO"]

    n_splits = row[1]["n_splits"]
    best_param = row[1][-1] # row with best params
    best_param_tuple = make_tuple(best_param)
    criterion = best_param_tuple['clf__criterion']
    splitter = best_param_tuple['clf__splitter']
    max_depth = best_param_tuple['clf__max_depth']
    max_leaf_nodes = best_param_tuple['clf__max_leaf_nodes']
    max_features = best_param_tuple['clf__max_features']

    # Apply best params to model to check accuracy
    clf = DecisionTreeClassifier(
        criterion = criterion,
        splitter = splitter,
        max_depth = max_depth,
        max_leaf_nodes =  max_leaf_nodes,
        max_features = max_features)

    estimator = Pipeline([("scl", StandardScaler()),
                          ("clf", clf)])
    
    skf = StratifiedKFold(n_splits = n_splits, shuffle = True)
    for i in range(1, 11):
        for index, (train_index, test_index) in enumerate(skf.split(X, y)):
            print(f"{i}'st run for split {index}")
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            start_time = time()
            estimator.fit(X_train, y_train)
            predictions = estimator.predict(X_test)
            end_time = time()
            total_time = timedelta(seconds=end_time-start_time)

            predict_data = {
                "database": dbname,
                "n_splits": f"{i}_{index}_{n_splits}",
                "confusion_matrix": confusion_matrix(y_test, predictions),
                "classification_report": classification_report(y_test, predictions),
                "model_accuracy": accuracy_score(y_test, predictions),
                "f1_score": f1_score(y_test, predictions),
                "precision_score": precision_score(y_test, predictions),
                "roc_score": roc_auc_score(y_test, predictions),
                "recall_score": recall_score(y_test, predictions),
                "log_loss": "na",
                "epoch_params": best_param,
                "mean_squared_error": mean_squared_error(y_test, predictions),
                "mean_absolute_error": mean_absolute_error(y_test, predictions),
                "train_test_duration": total_time,
                "train_size": len(X_train),
                "test_size": len(X_test)
            }

            all_predictions.append(predict_data)


# CSV Output results
model_utils.generate_output_csv(all_predictions, results_dir, "dtree_results")
print("FINISHED")